# Imports

In [ ]:
import numpy as np
import qutip as qtp
import math
import cmath
import matplotlib.pyplot as plt
from qutip import *
from tqdm.notebook import tqdm
from scipy import interpolate
from scipy.integrate import solve_ivp
from scipy.integrate import quad
%matplotlib inline
import matplotlib.colors as colors
import matplotlib as mpl
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
blues = mpl.colormaps['Blues']
import ROfunctions as ro
from ROfunctions import *
import pandas as pd

## Define Hilbert space, Hamiltonian terms, and global variables

In [ ]:
N = 25 #qubit Hilbert space
M = 10 #resonator Hilbert space

#qubit creation and annihilation operators
c = destroy(N)
cdag = create(N)

#resonator creation and annihilation operators
a = destroy(M)
adag = create(M)

rnum = adag * a
resonator_num = tensor(qeye(N), rnum)

#fluxonium energy parameters (GHz)
Ej = 4.75
Ec = 1.25
El = 1.5

#resonator frequency (GHz)
w = 7.0
H_lc = w * (adag * a + 1/2)

#qubit-resonator coupling strength (GHz)
g = 0.05
coupling1 = tensor(c, adag)
coupling2 = tensor(cdag, a)
H_i = g * (coupling1 + coupling2)

#reduced flux and charge operators for Hamiltonian
phi_naught = ((8 * Ec) / El)**(1/4)
n_op = (-1j / (math.sqrt(2) * phi_naught)) * (c - cdag)
phi_op = (phi_naught / math.sqrt(2)) * (c + cdag)
phi_op_HC = phi_op.dag()

Cterm = 4 * Ec * (n_op)**2
Lterm = (1/2) * El * phi_op**2

In [ ]:
n_readout = 10 #average cavity photon number
kappa = 2 * np.pi * 5 #resonator linewidth, 2*pi*MHz

## Simulate resonator dynamics with added quasistatic flux noise

In [ ]:
my_noise_values = np.array([0.0001, 0.001, 0.01]) #noise amplitudes
reps = 50 #number of random numbers generated
t_points = 2002 #time points
SNR_max_eff = np.zeros((len(my_noise_values), reps, t_points)) #max_eff denotes 100% measurement efficiency
error_max_eff = np.zeros((len(my_noise_values), reps, t_points))
SNR_low_eff = np.zeros((len(my_noise_values), reps, t_points)) #low_eff denotes 25% measurement efficiency
error_low_eff = np.zeros((len(my_noise_values), reps, t_points))

for p, noise in enumerate(tqdm(my_noise_values)):
    for m, j in enumerate(tqdm(range(reps))):
        num = np.random.randn(1)
        xi = noise * num[0] * 2 * np.pi

        sweet_spot = np.pi + xi
        RO_point = np.pi + (0.141*2*np.pi) + xi

        #calculate chi from sweet spot + offset to RO point + offset
        chi = []
        phi_ext = np.linspace(sweet_spot, RO_point, 101)

        for phi in phi_ext:
            N = 25
            M = 10
            Jterm = -Ej * ((1/2) * ((1j * (phi_op - phi)).expm()) + (1/2) * ((-1j * (phi_op_HC - phi)).expm()))
            H_flux = Jterm + Cterm + Lterm
            H_sys = tensor(H_flux, qeye(M)) + tensor(qeye(N), H_lc) + H_i
            states, energies, frequencies, chi_value, truncated_H = truncate_disshift(H_sys, resonator_num)
            chi.append(chi_value.real * 1000)

        #set dispersive shift of sweet spot and RO point
        chi_hfqp = 2 * np.pi * chi[0]
        chi_readout = 2 * np.pi * chi[-1]

        #use interpolation to get chi as a function of t to be used during 50 ns flux pulse
        chi_func_phi = interpolate.interp1d(phi_ext, chi)
        
        ramp_duration = 0.05 #rise time of flux pulse
        
        t = []
        for phi in phi_ext:
            time = (ramp_duration / (0.141*2*np.pi)) * (phi - np.pi)
            t.append(time)

        chi_func_t = interpolate.interp1d(t, chi, fill_value="extrapolate") #interpolation data

        #solve Langevin equation to get alpha during ramp up of flux pulse
        ep = math.sqrt(n_readout*(chi_readout**2 + (kappa)**2 / 4))

        def alphadot_0(alpha, time):
            dalpha0dt = -1j * 2 * np.pi * chi_func_t(time) * alpha - (1/2) * kappa * alpha + ep*(time/ramp_duration)**2
            return dalpha0dt

        def alphadot_1(alpha, time):
            dalpha1dt = 1j * 2 * np.pi * chi_func_t(time) * alpha - (1/2) * kappa * alpha + ep*(time/ramp_duration)**2
            return dalpha1dt

        alpha_init = [0+0j]
        t_ramp = np.linspace(0, ramp_duration, 101) #times during flux pulse ramping

        sol_alpha0 = solve_ivp(lambda time, alpha: alphadot_0(alpha, time), [t_ramp[0], t_ramp[-1]], alpha_init, t_eval=t_ramp)
        sol_alpha1 = solve_ivp(lambda time, alpha: alphadot_1(alpha, time), [t_ramp[0], t_ramp[-1]], alpha_init, t_eval=t_ramp)

        alpha0solution = sol_alpha0.y[0]
        alpha1solution = sol_alpha1.y[0]
        
        #get alpha out from alpha during flux pulse ramp
        alpha_out_0 = []
        alpha_out_1 = []

        #calculation of alpha_out from alpha and alpha_in
        for i, a in enumerate(alpha0solution):
            ep_td = ep*(t_ramp[i]/ramp_duration)**2
            aout0 = (-ep_td / math.sqrt(kappa)) + math.sqrt(kappa) * a
            alpha_out_0.append(aout0)

        for j, b in enumerate(alpha1solution):
            ep_td = ep*(t_ramp[j]/ramp_duration)**2
            aout1 = (-ep_td / math.sqrt(kappa)) + math.sqrt(kappa) * b
            alpha_out_1.append(aout1)
        
        aout0_func = interpolate.interp1d(t_ramp, alpha_out_0, fill_value="extrapolate")
        aout1_func = interpolate.interp1d(t_ramp, alpha_out_1, fill_value="extrapolate")

        rawSNR_ramp = [] #with flux pulse assisted readout

        for t in t_ramp:
            M_ramp = []
            tpts = np.linspace(0,t,1001)
            for i in tpts:
                alpha_zero = aout0_func(i)
                alpha_one = aout1_func(i)
                M_ramp.append(alpha_zero-alpha_one)
            SNRnumerator = np.sqrt(kappa)*abs(np.trapz(M_ramp))*np.diff(tpts)[0]
            SNRdenominator = math.sqrt(kappa * t)
            SNR_ramp = SNRnumerator / SNRdenominator
            rawSNR_ramp.append(SNR_ramp)

        #solve Langevin equation to get alpha after flux pulse has reached max amplitude
        def alphadot_0_fp(alpha, time): 
            dalpha0dt_fp = -1j * chi_readout * alpha - (1/2) * kappa * alpha + ep
            return dalpha0dt_fp

        def alphadot_1_fp(alpha, time):
            dalpha1dt_fp = 1j * chi_readout * alpha - (1/2) * kappa * alpha + ep
            return dalpha1dt_fp

        alpha_init_0 = [alpha0solution[-1]]
        alpha_init_1 = [alpha1solution[-1]]

        t_fp = np.linspace(ramp_duration, 1, 1901) #times after flux pulse, up to 1 us

        sol_alpha0_fp = solve_ivp(lambda time, alpha: alphadot_0_fp(alpha, time), [t_fp[0], t_fp[-1]], alpha_init_0, t_eval=t_fp)
        sol_alpha1_fp = solve_ivp(lambda time, alpha: alphadot_1_fp(alpha, time), [t_fp[0], t_fp[-1]], alpha_init_1, t_eval=t_fp)

        alpha0solution_fp = sol_alpha0_fp.y[0]
        alpha1solution_fp = sol_alpha1_fp.y[0]

        #get alpha out from alpha after flux pulse
        aout0 = []
        aout1 = []

        for a in alpha0solution_fp:
            alphaout0 = (-ep / math.sqrt(kappa)) + math.sqrt(kappa) * a
            aout0.append(alphaout0)

        for b in alpha1solution_fp:
            alphaout1 = (-ep / math.sqrt(kappa)) + math.sqrt(kappa) * b
            aout1.append(alphaout1)

        aout0_func_fp = interpolate.interp1d(t_fp, aout0, fill_value="extrapolate")
        aout1_func_fp = interpolate.interp1d(t_fp, aout1, fill_value="extrapolate")

        rawSNR_fp = []

        for t in t_fp:
            M = []
            tps = np.linspace(0,t,1001)
            for tp in tps:
                alpha_zero = aout0_func_fp(tp) if tp>0.05 else aout0_func(tp)
                alpha_one = aout1_func_fp(tp) if tp>0.05 else aout1_func(tp)
                M.append(alpha_zero-alpha_one)
            SNRnum = np.sqrt(kappa) * abs(np.trapz(M)*np.diff(tps)[0])
            SNRdenom = math.sqrt(kappa * t)
            SNR = SNRnum / SNRdenom
            rawSNR_fp.append(SNR)

        totalTime = np.concatenate((t_ramp, t_fp))
        totalRawSNR = np.concatenate((rawSNR_ramp, rawSNR_fp))

        SNR_max_eff[p,m] = totalRawSNR
        error_max_eff[p,m] = [math.erfc(s/2)/2 for s in totalRawSNR]

        eta = 0.25
        low_eff_SNR = np.sqrt(eta)*totalRawSNR
        SNR_low_eff[p,m] = low_eff_SNR
        error_low_eff[p,m] = [math.erfc(s/2)/2 for s in low_eff_SNR]

## Average over noise iterations

In [ ]:
SNR_max_eff_avg = np.mean(SNR_max_eff, axis=1) #max_eff refers to measurement efficiency of 100%
error_max_eff_avg = np.mean(error_max_eff, axis=1)
SNR_low_eff_avg = np.mean(SNR_low_eff, axis=1) #low_eff refers to measurement efficiency of 25%
error_low_eff_avg = np.mean(error_low_eff, axis=1)

## Export  data

In [ ]:
data = {'SNR_max_eff_ep0001': SNR_max_eff_avg[0],
        'SNR_max_eff_ep001': SNR_max_eff_avg[1],
        'SNR_max_eff_ep01': SNR_max_eff_avg[2],
        'error_max_eff_ep0001': error_max_eff_avg[0],
        'error_max_eff_ep001': error_max_eff_avg[1],
        'error_max_eff_ep01': error_max_eff_avg[2],
        'SNR_low_eff_ep0001': SNR_low_eff_avg[0],
        'SNR_low_eff_ep001': SNR_low_eff_avg[1],
        'SNR_low_eff_ep01': SNR_low_eff_avg[2],
        'error_low_eff_ep0001': error_low_eff_avg[0],
        'error_low_eff_ep001': error_low_eff_avg[1],
        'error_low_eff_ep01': error_low_eff_avg[2],
        'totalTime': totalTime}

df = pd.DataFrame(data)
df.to_csv('filepath/filename.csv')